In [42]:
%cd ~/eda-repo-3/

/home/park/eda-repo-3


In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# ▶️ 테마별 추천 자치구 데이터 추가
theme_data = {
    "건강 케어": ["강남구", "서초구", "송파구"],
    "스트레스 프리": ["강북구", "은평구", "서대문구"],
    "활발한 야외 활동": ["종로구", "중구", "서초구",],
    "문화 여가 동행": ["강남구", "마포구", "송파구"],
    "입양 봉사 관심": ["마포구", "노원구", "서대문구"]
}

# 각 자치구별 테마 개수 및 테마 목록 계산
district_theme_count = {}
district_themes = {}

for theme, districts in theme_data.items():
    for district in districts:
        if district not in district_theme_count:
            district_theme_count[district] = 0
            district_themes[district] = []
        district_theme_count[district] += 1
        district_themes[district].append(theme)

# 테마 개수에 따른 색상 함수 정의
def get_color_by_theme_count(count, min_count, max_count):
    if count == 0:
        return "gray"
    elif count == 1:
        return "#FFFFE0"  # 연한노랑
    elif count == 2:
        return "#FF8C00"  # 주황
    elif count == 3:
        return "#FF4500"  # 진한주황/빨강주황
    else:
        return "#DC143C"  # 빨강 (4개 이상)

# 각 자치구별 색상 계산
district_colors = {}
for district, count in district_theme_count.items():
    color = get_color_by_theme_count(count, 0, max(district_theme_count.values()))
    district_colors[district] = color

# 기존 데이터프레임에 테마 개수와 색상 정보 추가
df['theme_count'] = df['region'].map(district_theme_count).fillna(0)
df['color'] = df['region'].map(district_colors).fillna('gray')

# ▶️ GeoJSON 경로 지정
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'

# ▶️ GeoJSON 로드
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 스타일 함수 정의 (테마 개수에 따른 색상 적용)
def style_function(feature):
    region_name = feature['properties']['name']
    if region_name in district_colors:
        color = district_colors[region_name]
        return {'fillColor': color, 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)



# 테마별 마커 추가 (개선된 버전)
for district in district_themes.keys():
    coords = geo_df[geo_df['region'] == district]
    if not coords.empty:
        lat = coords['lat'].values[0]
        lng = coords['lng'].values[0]
        theme_count = district_theme_count[district]
        themes = district_themes[district]
        
        # marker_html 생성 부분 (레퍼런스 반영)
        # ex) 마포구 : 2개 테마
        #     문화 여가 동행
        #     입양 봉사 관심
        header = f"{district} : {theme_count}개 테마<br>"
        theme_divs = "".join([
            f"<div style='margin:2px 0;font-size:10px;'>{i+1}. {themes[i]}</div>"
            for i in range(len(themes))
        ])
        marker_html = header + theme_divs

        if theme_count == 1:
            # 1개 테마
            folium.Marker(
                location=[lat, lng],
                icon=folium.DivIcon(
                    html=f"""<div style="
                        font-size: 11px;
                        color: white;
                        background: #4ECDC4;
                        border-radius: 15px;
                        padding: 6px 8px;
                        font-weight: bold;
                        text-align: left;
                        border: 2px solid white;
                        box-shadow: 0 2px 5px rgba(0,0,0,0.3);
                        white-space: nowrap;
                        min-width: 100px;
                    ">{marker_html}</div>""",
                    icon_size=(80, 40),
                    icon_anchor=(40, 20)
                ),
                tooltip=marker_html.replace("<br>", "\n").replace("<div", "").replace("</div>", "")
            ).add_to(m)
            
        else:
            # 2개 이상 테마
            folium.Marker(
                location=[lat, lng],
                icon=folium.DivIcon(
                    html=f"""
                        <div style="
                            color: white;
                            background: #FF8C00;
                            border-radius: 15px;
                            padding: 6px 8px;
                            font-weight: bold;
                            text-align: left;
                            border: 2px solid white;
                            box-shadow: 0 2px 5px rgba(0,0,0,0.4);
                            line-height: 1.1;
                            min-width: 100px;
                        ">{marker_html}</div>
                    """,
                    icon_size=(110, 60 + (theme_count-2)*15),
                    icon_anchor=(55, 30)
                ),
                tooltip=marker_html.replace("<br>", "\n").replace("<div", "").replace("</div>", "")
            ).add_to(m)




# 지도 저장
m.save('RESULT/visualization/happy_home_folium_choropleth_themes.html')

# 결과 출력
print("지도가 생성되었습니다!")
print(f"\n자치구별 테마 개수:")
for district, count in sorted(district_theme_count.items()):
    themes_str = ", ".join(district_themes[district])
    print(f"- {district}: {count}개 테마 - {themes_str}")

m

지도가 생성되었습니다!

자치구별 테마 개수:
- 강남구: 2개 테마 - 건강 케어, 문화 여가 동행
- 강북구: 1개 테마 - 스트레스 프리
- 노원구: 1개 테마 - 입양 봉사 관심
- 마포구: 2개 테마 - 문화 여가 동행, 입양 봉사 관심
- 서대문구: 2개 테마 - 스트레스 프리, 입양 봉사 관심
- 서초구: 2개 테마 - 건강 케어, 활발한 야외 활동
- 송파구: 2개 테마 - 건강 케어, 문화 여가 동행
- 은평구: 1개 테마 - 스트레스 프리
- 종로구: 1개 테마 - 활발한 야외 활동
- 중구: 1개 테마 - 활발한 야외 활동
